In [2]:
from sklearn import datasets
import pandas as pd
import numpy as np

In [4]:
iris = datasets.load_iris()

In [5]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], 
      dtype='<U10')

In [6]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], 
      dtype='<U10')

In [7]:
type(iris)

sklearn.datasets.base.Bunch

In [8]:
df = pd.DataFrame(iris.data,columns = iris.feature_names)

In [9]:
df.head(n=10)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
5,5.4,3.9,1.7,0.4
6,4.6,3.4,1.4,0.3
7,5.0,3.4,1.5,0.2
8,4.4,2.9,1.4,0.2
9,4.9,3.1,1.5,0.1


In [10]:
df['class_label'] = iris.target

In [11]:
np.unique(df.class_label)

array([0, 1, 2])

In [12]:
df.dropna(how='all', inplace=True)

In [13]:
df.tail()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class_label
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2
149,5.9,3.0,5.1,1.8,2


In [14]:
#### Label Encoder



In [15]:
from sklearn.preprocessing import LabelEncoder

In [16]:
X = df.iloc[:,0:4].values

In [17]:
y = df['class_label'].values

In [18]:
enc = LabelEncoder()

In [19]:
label_encoder= enc.fit(y)
label_encoder

LabelEncoder()

In [20]:
y = label_encoder.transform(y) + 1

In [21]:
label_dict = {1 : 'setora', 2 : 'Versicolor', 3 : 'Virginica'}
label_dict

{1: 'setora', 2: 'Versicolor', 3: 'Virginica'}

In [22]:
np.set_printoptions(precision=4)

mean_vectors = []
for cl in range(1,4):
    mean_vectors.append(np.mean(X[y==cl], axis=0))    
    print("Mean vectors class %s: %s\n" %(cl, mean_vectors[cl-1]))
                        

Mean vectors class 1: [ 5.006  3.418  1.464  0.244]

Mean vectors class 2: [ 5.936  2.77   4.26   1.326]

Mean vectors class 3: [ 6.588  2.974  5.552  2.026]



In [23]:
S_W = np.zeros((4,4))
for cl,mv in zip(range(1,4), mean_vectors):
    class_sc_mat = np.zeros((4,4))
    for row in X[y==cl]:
        row,mv = row.reshape(4,1), mv.reshape(4,1)
        class_sc_mat += (row-mv).dot((row-mv).T)
    S_W += class_sc_mat
print('within-class scatter matrix:\n', S_W)

within-class scatter matrix:
 [[ 38.9562  13.683   24.614    5.6556]
 [ 13.683   17.035    8.12     4.9132]
 [ 24.614    8.12    27.22     6.2536]
 [  5.6556   4.9132   6.2536   6.1756]]


In [24]:
overall_mean = np.mean(X, axis=0)

S_B = np.zeros((4,4))
for i, mean_vec in enumerate(mean_vectors):
    n = X[y==i+1,:].shape[0]
    mean_vec = mean_vec.reshape(4,1)
    overall_mean = overall_mean.reshape(4,1)
    S_B += n * (mean_vec-overall_mean).dot((mean_vec-overall_mean).T)
    
print('between class scatter matrix:\n', S_B)
overall_mean

between class scatter matrix:
 [[  63.2121  -19.534   165.1647   71.3631]
 [ -19.534    10.9776  -56.0552  -22.4924]
 [ 165.1647  -56.0552  436.6437  186.9081]
 [  71.3631  -22.4924  186.9081   80.6041]]


array([[ 5.8433],
       [ 3.054 ],
       [ 3.7587],
       [ 1.1987]])

In [25]:
eig_vals, eig_vec = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))

for i in range(len(eig_vals)):
    eigvec_sc = eig_vec[:,i].reshape(4,1)
    print('\n EigenVector {}: \n{}'.format(i+1, eigvec_sc.real))
    print('\n EigenValues {}: {:.2e}'.format(i+1, eig_vals[i].real))
     


 EigenVector 1: 
[[-0.2049]
 [-0.3871]
 [ 0.5465]
 [ 0.7138]]

 EigenValues 1: 3.23e+01

 EigenVector 2: 
[[-0.009 ]
 [-0.589 ]
 [ 0.2543]
 [-0.767 ]]

 EigenValues 2: 2.78e-01

 EigenVector 3: 
[[-0.8049]
 [ 0.3474]
 [ 0.3405]
 [ 0.02  ]]

 EigenValues 3: 1.41e-15

 EigenVector 4: 
[[-0.8049]
 [ 0.3474]
 [ 0.3405]
 [ 0.02  ]]

 EigenValues 4: 1.41e-15


In [26]:
eig_vals, eig_vec = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))
eig_vec, eig_vals

(array([[-0.2049+0.j    , -0.0090+0.j    , -0.8049+0.j    , -0.8049-0.j    ],
        [-0.3871+0.j    , -0.5890+0.j    ,  0.3474+0.1145j,  0.3474-0.1145j],
        [ 0.5465+0.j    ,  0.2543+0.j    ,  0.3405+0.138j ,  0.3405-0.138j ],
        [ 0.7138+0.j    , -0.7670+0.j    ,  0.0200-0.2881j,  0.0200+0.2881j]]),
 array([  3.2272e+01 +0.0000e+00j,   2.7757e-01 +0.0000e+00j,
          1.4060e-15 +1.7359e-15j,   1.4060e-15 -1.7359e-15j]))

In [27]:
eig_pairs = [(np.abs(eig_vals[i]),eig_vec[:,i]) for i in range(len(eig_vals))]
eig_pairs = sorted(eig_pairs, key=lambda k:k[0], reverse=True)

In [28]:
print('Eigen values in decreasing order:\n')
for i in eig_pairs:
    print(i[0])

Eigen values in decreasing order:

32.2719577997
0.27756686384
2.23386208241e-15
2.23386208241e-15


In [29]:
print("Variance Explained\n")

eigv_sum = sum(eig_vals)
for i,j in enumerate(eig_pairs):
    print('eigen value {0:} {1:.2%}'.format(i+1, (j[0]/eigv_sum).real))

Variance Explained

eigen value 1 99.15%
eigen value 2 0.85%
eigen value 3 0.00%
eigen value 4 0.00%


In [30]:
### Now EigenVector W

W = np.hstack((eig_pairs[0][1].reshape(4,1), eig_pairs[1][1].reshape(4,1)))
print('Matrix W:\n',W.real)

Matrix W:
 [[-0.2049 -0.009 ]
 [-0.3871 -0.589 ]
 [ 0.5465  0.2543]
 [ 0.7138 -0.767 ]]


In [31]:
## Final Output

X_lda = X.dot(W)
assert X_lda.shape == (150,2), "This matrix is not 150x2 dimensional"

In [32]:
### LDA via scikit-learn

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

sklearn_LDA = LDA(n_components = 2)
X_lda_sklearn = sklearn_LDA.fit_transform(X,y)

In [36]:
###### Principal Component Analysis

from sklearn import datasets

In [39]:
iris = datasets.load_iris()

In [42]:
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['class-label'] = iris.target
df.tail()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),class-label
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2
149,5.9,3.0,5.1,1.8,2


In [43]:
x = df.iloc[:,0:4]

In [44]:
y = df.iloc[:,4]

In [50]:
from sklearn.preprocessing import StandardScaler
X_Std = StandardScaler().fit_transform(x)

In [59]:
import numpy as np

mean_vec = np.mean(X_Std, axis=0)
cov_mat = (X_Std - mean_vec).T.dot((X_Std - mean_vec)) / (X_Std.shape[0] - 1)
print('Covariance matrix: \n %s' %cov_mat)

Covariance matrix: 
 [[ 1.0067 -0.1101  0.8776  0.8234]
 [-0.1101  1.0067 -0.4233 -0.3589]
 [ 0.8776 -0.4233  1.0067  0.9692]
 [ 0.8234 -0.3589  0.9692  1.0067]]


In [62]:
print('Covariance matrix: \n %s' % np.cov(X_Std.T))

Covariance matrix: 
 [[ 1.0067 -0.1101  0.8776  0.8234]
 [-0.1101  1.0067 -0.4233 -0.3589]
 [ 0.8776 -0.4233  1.0067  0.9692]
 [ 0.8234 -0.3589  0.9692  1.0067]]


In [69]:
eig_vals, eig_vec = np.linalg.eig(cov_mat)
print('Eigen vectors: \n{}'.format(eig_vec))
print('Eigen Values: \n{}'.format(eig_vals))

Eigen vectors: 
[[ 0.5224 -0.3723 -0.721   0.262 ]
 [-0.2634 -0.9256  0.242  -0.1241]
 [ 0.5813 -0.0211  0.1409 -0.8012]
 [ 0.5656 -0.0654  0.6338  0.5235]]
Eigen Values: 
[ 2.9304  0.9274  0.1483  0.0207]


In [77]:
eig_pairs = [(np.abs(eig_vals[i]), eig_vec[:,i]) for i in range(len(eig_vals))]
eig_pairs = sorted(eig_pairs, key=lambda k: k[0], reverse=True)
for i in eig_pairs:
    print(i[0])

2.93035377559
0.927403621517
0.148342226482
0.0207460139956


In [79]:
print("Variance Explained\n")

eigv_sum = sum(eig_vals)
for i,j in enumerate(eig_pairs):
    print('eigen value {0:}: {1:.2%}'.format(i+1, (j[0]/eigv_sum).real))

Variance Explained

eigen value 1: 72.77%
eigen value 2: 23.03%
eigen value 3: 3.68%
eigen value 4: 0.52%


In [87]:
W = np.hstack((eig_pairs[0][1].reshape(4,1),eig_pairs[1][1].reshape(4,1)))

Y = X_Std.dot(W)

In [92]:
### Using scikit-learn

from sklearn.decomposition import PCA as sklearnPCA
sklearn_PCA = sklearnPCA(n_components=2)
Y_sklearn = sklearn_PCA.fit_transform(X_Std)